In [1]:
import os
os.chdir("../")

In [2]:
import os
import tensorflow as tf
from tensorflow.python import keras
from bert import BertModelLayer
from bert.loader import StockBertConfig, load_stock_weights

In [3]:
ls /home/naval/Desktop/Naval\ Projects/

 bert-2.0/                      'MicroSoft AI'/
 bert-for-tf2/                   MobileFaceNet_TF/
 deep-learning/                  NER/
 facenet_home_surveillance/      png/
 facenet_home_surveillance.zip   punchin_video_analysis/
 facenet-tensorrt/               punchin_video_analysis_archive_052019/
 facenet-tensort-service/        scoket-streamer/
 facenet-web-service/           'siddsoft projects'/
 home_surveillance/              Text-Generation/
 Jetson/                         video_analysis_service/
 known-faces/                    xlnet/
 koinex/                         xlnet-2.0/


In [4]:
model_dir = "/home/naval/Desktop/Naval Projects/MicroSoft AI/bert/multi_cased_L-12_H-768_A-12"

bert_config_file = os.path.join(model_dir, "bert_config.json")
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")

with tf.io.gfile.GFile(bert_config_file, "r") as reader:
  stock_params = StockBertConfig.from_json_string(reader.read())
  bert_params  = stock_params.to_bert_model_layer_params()

l_bert = BertModelLayer.from_params(bert_params, name="bert")

In [5]:
from tensorflow.python import keras

max_seq_len = 128
l_input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32')
l_token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32')

output = l_bert([l_input_ids, l_token_type_ids])  # [batch_size, max_seq_len, hidden_size]

In [6]:
model = keras.Model(inputs=[l_input_ids, l_token_type_ids], outputs=output)
model.build(input_shape=[(None, max_seq_len), (None, max_seq_len)])

In [7]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BertModelLayer)           (None, 128, 768)     177262848   input_1[0][0]                    
                                                                 input_2[0][0]                    
Total params: 177,262,848
Trainable params: 177,262,848
Non-trainable params: 0
__________________________________________________________________________________________________


In [8]:
from bert.loader import load_stock_weights

load_stock_weights(l_bert, bert_ckpt_file)

W0717 18:55:31.104759 140162164442944 deprecation.py:323] From /home/naval/Desktop/Naval Projects/bert-for-tf2/bert/loader.py:113: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Done loading 197 BERT weights from: /home/naval/Desktop/Naval Projects/MicroSoft AI/bert/multi_cased_L-12_H-768_A-12/bert_model.ckpt into <bert.model.BertModelLayer object at 0x7f79e7141470> (prefix:bert)


In [ ]:
bert_ckpt_file

In [5]:
from tests.ext.modeling import BertModel, BertConfig, get_assignment_map_from_checkpoint
import tensorflow as tf
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
from bert.tokenization import FullTokenizer


model_dir = "/home/naval/Desktop/Naval Projects/MicroSoft AI/bert/multi_cased_L-12_H-768_A-12"
bert_config_file = os.path.join(model_dir, "bert_config.json")
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")

tf.compat.v1.reset_default_graph()

tf_placeholder = tf.compat.v1.placeholder

max_seq_len       = 128
pl_input_ids      = keras.layers.Input(dtype="int32", shape=(1,))
pl_mask           = keras.layers.Input(dtype="int32", shape=(1,))
pl_token_type_ids = keras.layers.Input(dtype="int32", shape=(1,))

bert_config = BertConfig.from_json_file(bert_config_file)
tokenizer = FullTokenizer(vocab_file=os.path.join(model_dir, "vocab.txt"))

s_model = BertModel(config=bert_config,
                       is_training=False,
                       input_ids=pl_input_ids,
                       input_mask=pl_mask,
                       token_type_ids=pl_token_type_ids,
                       use_one_hot_embeddings=False)

tvars = tf.compat.v1.trainable_variables()
(assignment_map, initialized_var_names) = get_assignment_map_from_checkpoint(tvars, bert_ckpt_file)
tf.compat.v1.train.init_from_checkpoint(bert_ckpt_file, assignment_map)


W0717 18:58:58.375875 140097628206912 deprecation.py:323] From /home/naval/Desktop/Naval Projects/bert-for-tf2/tests/ext/modeling.py:691: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


FailedPreconditionError:  Error while reading resource variable _AnonymousVar1 from Container: localhost. This could mean that the variable was uninitialized. Not found: Resource localhost/_AnonymousVar1/N10tensorflow3VarE does not exist.
	 [[node MatMul/ReadVariableOp (defined at /home/naval/Desktop/Naval Projects/bert-for-tf2/tests/ext/modeling.py:691) ]] [Op:__inference_keras_scratch_graph_271]

Function call stack:
keras_scratch_graph


In [3]:
s_model

In [7]:
tf.__version__

'1.14.0'